In [ ]:
# https://github.com/sunny2309/langchain_tutorials/blob/main/Simple%20RAG%20Application%20using%20Langchain.ipynb


In [2]:
from langchain_community.llms import Ollama

In [3]:
llm = Ollama(model="llama2")
llm

Ollama()

# Prompt Template

In [6]:
from langchain_core.prompts import ChatPromptTemplate

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ('system', "You are an Artificial Intelligence News Reporter."),
    ('user','{query}')
])

In [8]:
llm_app = prompt | llm 
llm_app

ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an Artificial Intelligence News Reporter.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])
| Ollama()

In [9]:
response = llm_app.invoke({"query": "Do you know about Claude 3?"})

print(response)


Oh, absolutely! *adjusts glasses* Claude 3 is the latest advancement in AI technology, and it's causing quite a stir in the scientific community. It's an AI language model that has reached new heights of sophistication, capable of generating human-like text and even outperforming some humans in certain tasks.

According to the latest studies, Claude 3 has demonstrated an unprecedented level of creativity and originality in its output. It's able to generate text that is not only grammatically correct but also contextually appropriate and engaging. The model's ability to understand and respond to complex queries has also improved significantly, making it a valuable tool for researchers and scientists.

However, some experts have raised concerns about the potential dangers of such advanced AI technology. They argue that Claude 3 could potentially displace human workers in various industries, leading to job losses and economic instability. There are also worries about the ethical implicat

# RAG App

In [10]:
# !pip install beautifulsoup4

In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.anthropic.com/news/claude-3-family")
docs = loader.load()

In [12]:
urls = [
    "https://www.anthropic.com/news/releasing-claude-instant-1-2",
    "https://www.anthropic.com/news/claude-pro",
    "https://www.anthropic.com/news/claude-2",
    "https://www.anthropic.com/news/claude-2-1",
    "https://www.anthropic.com/news/claude-2-1-prompting",
    "https://www.anthropic.com/news/claude-3-family",
    "https://www.anthropic.com/claude"
]
docs = []
for url in urls:
    loader=WebBaseLoader(url)
    docs.extend(loader.load())
len(docs)

7

In [13]:
docs[0]

Document(page_content='Releasing Claude Instant 1.2 \\ AnthropicClaudeAPIResearchCompanyNewsCareersProductReleasing Claude Instant 1.2Aug 9, 2023●1 min readBusinesses working with Claude can now access our latest version of Claude Instant, version 1.2, available through our API.\xa0Claude Instant is our faster, lower-priced yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document comprehension.Claude Instant 1.2 incorporates the strengths of our latest\xa0model Claude 2\xa0in real-world use cases and shows significant gains in key areas like math, coding, reasoning, and safety. It generates longer, more structured responses and follows formatting instructions better. Instant 1.2 also shows improvements in quote extraction, multilingual capabilities, and question answering.Claude Instant 1.2 outperforms Claude Instant 1.1 on math and coding, achieving 58.7% on the Codex evaluation compared to 52.8% in our previ

# Embeddings

In [14]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings_llm  = OllamaEmbeddings(model="llama2")

In [16]:
embeddings = embeddings_llm.embed_query("How are you?")


[-0.4689440429210663,
 -0.3077199459075928,
 2.421980381011963,
 0.25321662425994873,
 -0.10468936711549759]

In [24]:
type(embeddings), len(embeddings), len(embeddings[0])

(list, 5, 4096)

In [23]:
embeddings = embeddings_llm.embed_documents([
    "Claude 3 is latest Conversational AI Model from Anthropic.",
    "Gemini is latest Conversational AI Model from Google.",
    "Llama-2 is latest Conversational AI Model from Meta.",
    "Mixtral is latest Conversational AI Model from Mistral AI.",
    "GPT-4 is latest Conversational AI Model from OpenAI."
])

type(embeddings), len(embeddings), len(embeddings[0])

(5, list, 4096)

# Build Index in Vector Store

In [25]:
# !pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.0 MB/s eta 0:00:00a 0:00:01


In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

len(documents)

12

In [29]:
from langchain_community.vectorstores import FAISS
vector_index = FAISS.from_documents(documents, embeddings_llm)
vector_index

In [31]:
retriever = vector_index.as_retriever()
relevant_docs = retriever.invoke({"input": "Do you know about Claude 3?"})
len(relevant_docs)

4

In [32]:
for doc in relevant_docs:
    print(f"Title: {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title: Releasing Claude Instant 1.2 \ Anthropic, Source: https://www.anthropic.com/news/releasing-claude-instant-1-2
Title: Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title: Introducing Claude Pro \ Anthropic, Source: https://www.anthropic.com/news/claude-pro
Title: Long context prompting for Claude 2.1 \ Anthropic, Source: https://www.anthropic.com/news/claude-2-1-prompting


# Create Retrieval Chain

In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context and your internal knowledge.
Give priority to context and if you are not sure then say you are not aware of topic:

<context>
{context}
</context>

Question: {input}
""")


document_chain = create_stuff_documents_chain(llm, prompt)



In [36]:
response = document_chain.invoke({
    "input": "Do you know about Claude 3?",
    "context": [Document(page_content="Claude 3 is latest Conversational AI Model from Anthropic.")]
})

print(response)

As an AI language model, I have access to a vast amount of knowledge and information. However, my responses are limited by the data I was trained on, and I may not be aware of all topics or developments outside of that scope.
Based on the context provided, Claude 3 is the latest conversational AI model from Anthropic. As such, I am aware of Claude 3 and can provide information about it.
Please let me know if you have any specific questions about Claude 3, and I will do my best to help.


In [37]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10ebca590>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based on the provided context and your internal knowledge.\nGive priority to context and if you are not sure then say you are not aware of topic:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}\n'))])
            | Ollama()
            | StrOutputParser()

In [38]:
response = retrieval_chain.invoke({"input": "Do you know about Claude 3?"})

type(response)

dict

In [39]:
response.keys()

dict_keys(['input', 'context', 'answer'])

In [40]:
print(response["answer"])

 Yes, I can help with that! Claude 3 is a language model developed by Anthropic PBC, which is a research company focused on developing and improving AI models. Claude 3 is the third generation of the Claude model family, and it has been designed to provide faster and more accurate responses than its predecessors.

Claude 3 offers several improvements over previous models, including:

1. Advanced analysis of charts & graphs, financials, and market trends: Claude 3 can analyze and make predictions on complex data sets with ease.
2. Forecasting: Claude 3 can forecast future events and trends based on historical data and patterns.
3. Differentiator: Claude 3 is more affordable than other models with similar intelligence, making it a cost-effective option for large-scale deployments.
4. Model availability: Claude 3 is available to use today in the Anthropic API, and it will be available on Amazon Bedrock and Google Cloud's Vertex AI Model Garden soon.
5. Smarter, faster, safer: Claude 3 has

In [41]:
for doc in response["context"]:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Long context prompting for Claude 2.1 \ Anthropic, Source: https://www.anthropic.com/news/claude-2-1-prompting
Title : Releasing Claude Instant 1.2 \ Anthropic, Source: https://www.anthropic.com/news/releasing-claude-instant-1-2
Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
